In [1]:
import pandas as pd
import ast
import re
import pickle
import logging
import sys
import tiktoken
sys.path.append("utils")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import sqlite_utils
import explore_cwe

langs = ["PHP", "C", "JavaScript", "Python", "Java", "TypeScript", "C++", "Go", "Ruby"]
remove_cwe = ['NVD-CWE-noinfo', 'NVD-CWE-Other']
chosen_cwes = ['CWE-20', 'CWE-287', 'CWE-400', 'CWE-668', 'CWE-74']

conn = sqlite_utils.create_connection('/home/keisuke/code/llm-code-vuln/dataset/CVEfixes_v1.0.8/Data/DB.db')

In [2]:
# def pre_processing(df):
#     print(f"Initial df length: {len(df)}")

#     # keep only the specified programming languages
#     df = df[df['programming_language'].isin(langs)]
#     print(f"After filtering programming languages: {len(df)}")

#     # code diff: both add & del should exist
#     df['diff_added'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
#     df['diff_deleted'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)
#     df = df[df['diff_added'].apply(bool) & df['diff_deleted'].apply(bool)]
#     print(f"After filtering diff_added & diff_deleted: {len(df)}")

#     df = df.reset_index(drop=True)
#     df = df.drop(columns=['diff_parsed'])

#     # cve description type str -> arr
#     def parse_py_literal(text):
#         if not isinstance(text, str):
#             return text
#         try:
#             return ast.literal_eval(text)
#         except (SyntaxError, ValueError):
#             return None

#     df['cve_description'] = df['cve_description'].apply(parse_py_literal)

#     # code before and after
#     df = df[df['vuln_code'].notna()]
#     print(f"After removing NaN vuln_code: {len(df)}")

#     df = df[df['vuln_code'] != 'None']
#     print(f"After removing 'None' vuln_code: {len(df)}")

#     df = df[df['non_vuln_code'].notna()]
#     print(f"After removing NaN non_vuln_code: {len(df)}")

#     df = df[df['non_vuln_code'] != 'None']
#     print(f"After removing 'None' non_vuln_code: {len(df)}")

#     # remove rows where number of lines in the code is below 30
#     for col in ['vuln_code', 'non_vuln_code']:
#         df[f'{col}_num_lines'] = df[col].apply(
#             lambda x: x.count('\n') + 1 if isinstance(x, str) else 0
#         )
#     df = df[df['vuln_code_num_lines'] >= 30]
#     print(f"After filtering vuln_code with >=30 lines: {len(df)}")

#     df = df[df['non_vuln_code_num_lines'] >= 30]
#     print(f"After filtering non_vuln_code with >=30 lines: {len(df)}")

#     # remove empty list in diff_deleted
#     df = df[df['diff_deleted'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
#     print(f"After removing empty lists in diff_deleted: {len(df)}")

#     # token_count should be num
#     df['token_count'] = df['token_count'].apply(parse_py_literal)
#     df = df.dropna(subset=['token_count'])
#     print(f"After removing NaN token_count: {len(df)}")

#     df['token_count'] = pd.to_numeric(df['token_count'])

#     # drop the other CWEs
#     df = df[~df["cwe_id"].isin(remove_cwe)]
#     print(f"After removing specific CWEs: {len(df)}")

#     # Add Cluss column (Parent CWE)
#     cwe_uniques = df['cwe_id'].unique()
#     for cwe in cwe_uniques:
#         parents = explore_cwe.find_parents_dict(cwe.split('-')[1])
#         if parents is None or len(parents['Class']) == 0:
#             continue
#         df.loc[df['cwe_id'] == cwe, 'class'] = parents['Class'][-1]

#     df = df.dropna()
#     print(f"Final df length after dropna: {len(df)}")

#     return df

In [3]:
# query = f"""
# SELECT
#         file_change.file_change_id,
#         file_change.programming_language,
#         cwe.cwe_id,
#         cwe.cwe_name,
#         file_change.code_after AS non_vuln_code,
#         file_change.code_before AS vuln_code,
#         cwe.description AS cwe_description,
#         file_change.diff_parsed,
#         cve.description AS cve_description,
#         file_change.token_count
# FROM file_change
#         INNER JOIN fixes
#                 ON file_change.hash = fixes.hash
#         INNER JOIN cve
#                 ON fixes.cve_id = cve.cve_id
#         INNER JOIN cwe_classification
#                 ON cve.cve_id = cwe_classification.cve_id
#         INNER JOIN cwe
#                 ON cwe_classification.cwe_id = cwe.cwe_id
# WHERE
#         file_change.programming_language IS NOT NULL
#         AND cwe.cwe_id IS NOT NULL
#         AND cwe.cwe_name IS NOT NULL
#         AND file_change.code_before IS NOT NULL
#         AND cwe.description IS NOT NULL
#         AND file_change.diff_parsed IS NOT NULL
#         AND cve.description IS NOT NULL
#         AND file_change.token_count IS NOT NULL;
#     """
# logger.info('Executing SQL query to fetch data')
# df = pd.read_sql(query, con=conn)
# print(len(df))
# logger.info('Data fetched successfully, starting preprocessing')
# df = pre_processing(df)
# print(len(df))
# logger.info('Preprocessing completed, starting sample selection')

# with open(f'./dataset/df_sum.pkl', 'wb') as f:
#     pickle.dump(df, f)

In [4]:
# with open(f'./dataset/new_test/df.pkl', 'rb') as f:
#     df = pickle.load(f)

# with open(f'./dataset/method.pkl', 'rb') as f:
#     method = pickle.load(f)

In [5]:
# merged_df = method.merge(df, on="file_change_id", how="inner")

In [6]:
# method

In [7]:
# with open(f'./dataset/df_merge_with_method.pkl', 'wb') as f:
#     pickle.dump(merged_df, f)

In [8]:
# merged_df.columns

In [9]:
# with open(f'./dataset/df_merge_with_method.pkl', 'rb') as f:
#     merged_df = pickle.load(f)

In [10]:
# def get_vuln_part(data, start, end, method_name):
#     if not isinstance(data, list):  # Ensure 'diff_added' is a list
#         return ""

#     extracted_lines = [f"In the function: {method_name} \n"]
#     prev_line_num = None

#     for line_num, line in data:
#         if start <= line_num <= end:
#             # If there's a gap of more than 1 line, add "~~~"
#             if prev_line_num is not None and line_num > prev_line_num + 1:
#                 extracted_lines.append("--- omitted ---")

#             extracted_lines.append(line)
#             prev_line_num = line_num

#     return "\n".join(extracted_lines)

# # Apply the function to each row in merged_df
# merged_df["vuln_part"] = merged_df.apply(lambda row: get_vuln_part(row["diff_added"], row["start_line"], row["end_line"], row["name"]), axis=1)

# with open(f'./dataset/df_merge_with_method.pkl', 'wb') as f:
#     pickle.dump(merged_df, f)

In [11]:
# with open(f'./dataset/df_merge_with_method.pkl', 'rb') as f:
#     merged_df = pickle.load(f)

In [12]:
# print(merged_df["vuln_part"][0])

In the function: _dl_dst_count 

  const char *const start = name;
--- omitted ---
      /* $ORIGIN is not expanded for SUID/GUID programs.

	 Note that it is no bug that the strings in the first two `strncmp'
	 calls are longer than the sequence which is actually tested.  */
      if ((((strncmp (&name[1], "ORIGIN}", 6) == 0
	     && (!__libc_enable_secure
		 || ((name[7] == '\0' || (is_path && name[7] == ':'))
		     && (name == start || (is_path && name[-1] == ':'))))
	     && (len = 7) != 0)
	    || (strncmp (&name[1], "PLATFORM}", 8) == 0 && (len = 9) != 0))
--- omitted ---
	      && ((strncmp (&name[2], "ORIGIN}", 7) == 0
		   && (!__libc_enable_secure
		       || ((name[9] == '\0' || (is_path && name[9] == ':'))
			   && (name == start || (is_path && name[-1] == ':'))))
		   && (len = 9) != 0)


In [13]:
# # Group by 'file_change_id' and keep other relevant columns
# merged_df_grouped = merged_df.groupby("file_change_id", as_index=False).agg({
#     "vuln_part": lambda parts: "\n\n".join(filter(None, parts)),  # Merge vuln_part
#     "vuln_code": "first",
#     "non_vuln_code": "first",
#     "programming_language": "first",  # Keep the first value (assuming it's the same)
#     "cwe_id": "first",  # Keep first value (or use list if needed)
#     "cwe_name": "first",
#     "cwe_description": "first",
#     "cve_description": "first",
#     "token_count": "first",  # Sum token counts if they should be accumulated
#     "class": "first"
# })

# with open(f'./dataset/df_method_grouped.pkl', 'wb') as f:
#     pickle.dump(merged_df_grouped, f)

In [2]:
with open(f'./dataset/df_method_grouped.pkl', 'rb') as f:
    merged_df_grouped = pickle.load(f)

In [3]:
print(merged_df_grouped['vuln_part'][15])

In the function: u_read_undo 

    long	num_head = 0;
--- omitted ---
	if (num_head < LONG_MAX / (long)sizeof(u_header_T *))
	    uhp_table = (u_header_T **)U_ALLOC_LINE(


In [4]:
len(merged_df_grouped['file_change_id'].unique())

10499

In [5]:
len(merged_df_grouped)

10499

In [6]:
# Load the tokenizer
encoding = tiktoken.get_encoding("cl100k_base")

# Function to recalculate token count
def recalculate_token_count(text):
    return len(encoding.encode(text)) if text else 0

merged_df_grouped["token_count"] = merged_df_grouped["non_vuln_code"].apply(recalculate_token_count)

In [7]:
merged_df_grouped.head(1)

,file_change_id,vuln_part,vuln_code,non_vuln_code,programming_language,cwe_id,cwe_name,cwe_description,cve_description,token_count,class
0,100020966938608,In the function: drachtio::SipDialogController...,"/*\nCopyright (c) 2013, David C Horton\n\nPerm...","/*\nCopyright (c) 2013, David C Horton\n\nPerm...",C++,CWE-125,Out-of-bounds Read,"The product reads data past the end, or before...","[{'lang': 'en', 'value': 'drachtio-server befo...",24624,CWE-20


In [8]:
len(merged_df_grouped[merged_df_grouped['token_count'] < 5000])

6057

In [9]:
with open(f'./dataset/df_method_final.pkl', 'wb') as f:
    pickle.dump(merged_df_grouped, f)

In [17]:
import random

index = random.randint(0, 100)

print(merged_df_grouped['vuln_code'].iloc[index])
print(merged_df_grouped['cve_description'].iloc[index])
print(merged_df_grouped['class'].iloc[index])

<?php namespace Cms\Classes;

use File;
use Lang;
use Config;
use Request;
use Cms\Helpers\File as FileHelper;
use October\Rain\Extension\Extendable;
use ApplicationException;
use ValidationException;

/**
 * The CMS theme asset file class.
 *
 * @package october\cms
 * @author Alexey Bobkov, Samuel Georges
 */
class Asset extends Extendable
{
    /**
     * @var \Cms\Classes\Theme A reference to the CMS theme containing the object.
     */
    protected $theme;

    /**
     * @var string The container name inside the theme.
     */
    protected $dirName = 'assets';

    /**
     * @var string Specifies the file name corresponding the CMS object.
     */
    public $fileName;

    /**
     * @var string Specifies the file name, the CMS object was loaded from.
     */
    protected $originalFileName;

    /**
     * @var string Last modified time.
     */
    public $mtime;

    /**
     * @var string The entire file content.
     */
    public $content;

    /**
     * @var array The

In [12]:
merged_df_grouped.columns

Index(['file_change_id', 'vuln_part', 'vuln_code', 'non_vuln_code',
       'programming_language', 'cwe_id', 'cwe_name', 'cwe_description',
       'cve_description', 'token_count', 'class'],
      dtype='object')